# Прогнозирование оттока клиентов

Оператор связи «Ниединогоразрыва.ком» хочет научиться прогнозировать отток клиентов. Если выяснится, что пользователь планирует уйти, ему будут предложены промокоды и специальные условия. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и договорах.

**Цель**: модель должна предсказывать, уйдет ли клиент или останется. В случае возможного ухода клиента, предложить особые условия обслуживания, которые позволит сохранить его (клиента) и сэкономить на привлечении нового клиента. В качестве основной метрики выбран ROC-AUC, оценка которой в идеале должна быть выше 0.85. На создание прототипа модели отведено 4 дня. Еще 4 дня на подготовку отчета.

**План работы:**
1. Изучить данные и сделать предобработку
    * Разобраться с пустыми строками
    * Соединить таблицы (можно после анализа данных)
2. Провести исследовательский анализ данных
3. Подобрать 3 модели (возможны изменения) и выбрать лучшую:
    * Логистическая регрессия
    * Метод случайного леса
    * KNN-метод
4. Сделать отчет по проведенной работе

_Часть предобработки данных я сделала до встречи по зуму, поэтому решила оставить как есть, только добавила план._ <br>
**Вопросов нет** (пока что)

## Предобработка и Исследовательский анализ данных

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, auc, roc_curve, confusion_matrix, accuracy_score
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier, Pool

In [4]:
contract_df = pd.read_csv('/datasets/final_provider/contract.csv')
personal_df = pd.read_csv('/datasets/final_provider/personal.csv')
internet_df = pd.read_csv('/datasets/final_provider/internet.csv')
phone_df = pd.read_csv('/datasets/final_provider/phone.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/final_provider/contract.csv'

### Контракты

In [ ]:
display(contract_df.head())

**Целевой признак**: целевым признаком станет дата окончания договора `EndDate`. Таргет следует сделать бинарным.

In [ ]:
contract_df.info()

`TotalCharges`: необходимо поменять на тип данных `float64` <br>
`BeginDate` и `EndDate`: необходимо поменять на тип данных `datetime`

In [ ]:
contract_df['BeginDate'] = pd.to_datetime(contract_df['BeginDate'], format='%Y-%m-%d')

In [ ]:
contract_df['BeginDate'].sort_values(ascending=True)

In [ ]:
# Замена типа данных с str на float
# contract_df['TotalCharges'] = contract_df['TotalCharges'].astype('float64')

# ---> ValueError: could not convert string to float: ''

Ошибка. Надо посмотреть первые пять строк с пустыми значениями.

In [ ]:
display(contract_df[contract_df['TotalCharges'] == ' '].head())

По идее, если дата начала договора 2020-02-01, то можно понять, почему в колонке `TotalCharges` пустые строки — клиент еще не успел оплатить подписку и одного месяца. _Эти пробелы можно заполнить нулями._

In [ ]:
# Замена пустых строк нулями
contract_df['TotalCharges'] = contract_df['TotalCharges'].replace(' ', 0.0)

In [ ]:
# Замена типа данных с str на float
contract_df['TotalCharges'] = contract_df['TotalCharges'].astype('float64')

С замененным типом данных и пропусками можно посмотреть на обновленную таблицу:

In [ ]:
display(contract_df[contract_df['TotalCharges'] == 0.0].head())

In [ ]:
contract_df.info()

Типы данных стоят те, какие и должны быть.

In [5]:
print('Доля действующих договоров:',
      contract_df[contract_df['EndDate'] == 'No']['EndDate'].count() / contract_df['EndDate'].count())

NameError: name 'contract_df' is not defined

In [ ]:
display(contract_df.describe())

**Ежемесячные платежи**: Медиана от среднего слабо отличаются, поэтому можно сказать, что выбросов практически нет.

**Суммарные платежи**: Здесь можно увидеть, как сильно отличается среднее в большую сторону от медианы. В этом признаке выбросы сильные.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(16, 5))
ax1 = sns.boxplot(contract_df['MonthlyCharges'], ax=ax[0]);
ax1.set_title('Monthly Charges')

ax2 = sns.boxplot(contract_df['TotalCharges'], ax=ax[1]);
ax2.set_title('Total Charges')

На втором боксплоте выбросов нет, но смещение вправо, конечно, наблюдается.

### Персональные данные

In [ ]:
display(personal_df.head())

In [6]:
personal_df.info()

NameError: name 'personal_df' is not defined

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=1, figsize=(16, 4))

ax0 = sns.histplot(personal_df['gender'], ax=ax[0])
ax1 = sns.histplot(personal_df['SeniorCitizen'], ax=ax[1])
ax2 = sns.histplot(personal_df['Partner'], ax=ax[2])
ax3 = sns.histplot(personal_df['Dependents'], ax=ax[3])

* У нас наблюдается сбалансированные признаки `gender` и `Partner`.
* А признаки `SeniorCitizen` и `Dependents` уже несбалансированы.
* Также нужно поменять название `gender` на `Gender`.

Признаки `SeniorCitizen` и `Dependets` можно попробовать сбалансировать.

In [ ]:
personal_df = personal_df.rename({'gender': 'Gender'}, axis=1)

### Интернет услуги

In [ ]:
internet_df.head()

In [ ]:
internet_df.info()

В этой таблице содержатся ID пользователей, у которых активна мининум одна услуга. Поэтому при присоединении этой таблицы к основной, пропуски заполним значениями `No`.

Посмотрим также на распределения классов.

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=2, figsize=(16, 9))

ax0 = sns.histplot(internet_df['InternetService'], ax=ax[0, 0])
ax1 = sns.histplot(internet_df['OnlineSecurity'], ax=ax[0, 1])
ax2 = sns.histplot(internet_df['OnlineBackup'], ax=ax[0, 2])
ax3 = sns.histplot(internet_df['DeviceProtection'], ax=ax[0, 3])
ax4 = sns.histplot(internet_df['TechSupport'], ax=ax[1, 0])
ax5 = sns.histplot(internet_df['StreamingTV'], ax=ax[1, 1])
ax6 = sns.histplot(internet_df['StreamingMovies'], ax=ax[1, 2])

fig.delaxes(ax[1, 3])

* Здесь наблюдается дисбаланс классов на 50% у `OnlineSecurity` и `TechSupport`
* У оставшихся классов с этим более менее нормально.

### Подключение нескольких линий

In [ ]:
phone_df.head()

In [ ]:
phone_df.info()

Логично предположить здесь то же самое (про заполнение пропусков значениями `No`). Но лучше проверить основную таблицу `contact_df` на наличие дубликатов в колонке `customerID`. Может в основной таблице одни и те же ID могли заключать несколько договоров в разные промежутки времени.

In [7]:
print('Кол-во дублей:', contract_df['customerID'].duplicated().sum())

NameError: name 'contract_df' is not defined

Догадка не подтвердилась. Все ID пользователей разные.

### Соединение таблиц

In [ ]:
data = contract_df.merge(personal_df, how='left', on='customerID')
data = data.merge(internet_df, how='left', on='customerID')
data = data.merge(phone_df, how='left', on='customerID')

In [ ]:
data.head()

In [ ]:
data.info()

Теперь все пропуски заполним `No`, т.к. если ID клиента отсутствовало в той или иной таблице, то можно предположить, что теми услугами он не пользовался.

In [ ]:
data = data.fillna('No')

In [ ]:
print('Кол-во пропусков в датасете:', data.isna().sum())

**Теперь создадим бинарный таргет**

In [ ]:
data['isChurn'] = list(map(lambda x: 0 if (x == 'No') else 1, data['EndDate']))

In [ ]:
data['isChurn'].head()

Отлично, теперь проверим признаки на мультиколлениарность и удалим лишние. Будем использовать VIF, для этого нужно перевести данные в численные типы. Используем разные виды кодирования.

Теперь создадим синтетический признак, который будет кол-во дней, сколько провел клиент с компанией.

In [8]:
data['EndDate'] = data['EndDate'].replace('No', '2020-02-01')

data['EndDate'] = pd.to_datetime(data['EndDate'], format='%Y-%m-%d')

NameError: name 'data' is not defined

In [ ]:
data['Days'] = (data['EndDate'] - data['BeginDate']).dt.days

### Проверка на мультиколлинеарность

In [ ]:
data = data.drop(['BeginDate', 'EndDate', 'customerID'], axis=1)

Чтобы не множить дамми колонки через OHE, используем LabelEncoder для тех колонок, где всего два категориальных признака. LabelEncoder используется т.к. через него есть возможность передать одномерный массив, результатом которого можно будет датафрейм, а не вектор, как у OrdinalEncoder. А это удобно для дальнейшей работы.

In [ ]:
print(data['Type'].unique())
print(data['InternetService'].unique())
print(data['PaymentMethod'].unique())

У колонки `Type` можно обнаружить порядок, поэтому для него тоже используем LabelEncoder.

In [9]:
features_ordinal = list(data.drop(['PaymentMethod', 'SeniorCitizen', 'InternetService', 'isChurn', 'MonthlyCharges',
                                   'TotalCharges'], axis=1).columns)

NameError: name 'data' is not defined

In [ ]:
# Копиравоние датафрейма в новую переменную, чтобы потом была возможность работать со старым датафреймом
data_enc = data.copy()

In [ ]:
for col in features_ordinal:
    data_enc[col] = LabelEncoder().fit_transform(data[col])
    
data_enc.head() 

In [ ]:
data_dummies = pd.get_dummies(data[['PaymentMethod', 'InternetService']], drop_first=True)

data_dummies.head()

In [ ]:
# Соединение таблиц по индексам
data_enc = data_enc.join(data_dummies)

# Удаление object колонок
data_enc = data_enc.drop(['PaymentMethod', 'InternetService'], axis=1)

In [ ]:
data_enc.head()

In [ ]:
# Выделение нужных признаков в закодированном датафрейме
features = list(data_enc.drop(['isChurn'], axis=1).columns)

In [ ]:
# подсчет vif для всех подаваемых признаков
def compute_vif(features):
    
    X = data_enc[features]
    X['intercept'] = 1
    
    # Создание датафрейма для хранения vif-значений
    vif = pd.DataFrame()
    vif['Variable'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['Variable'] != 'intercept']
    return vif

In [ ]:
compute_vif(features).sort_values('VIF', ascending=False)

Можно увидеть сильную зависимость от признака `MonthlyCharges`. Будем удалять последовательно признаки, начиная с этого.

In [ ]:
features.remove('MonthlyCharges')

compute_vif(features).sort_values('VIF', ascending=False)

In [ ]:
features.remove('TotalCharges')

In [ ]:
compute_vif(features).sort_values('VIF', ascending=False)

Дальше удалять признаки не нужно, т.к. никакие из них не превышают значения `5`. Можно предположить, что утечки данных не должно произойти.

In [ ]:
data_enc_linear = data_enc.drop(['MonthlyCharges', 'TotalCharges'], axis=1)

In [ ]:
sns.histplot(data['isChurn'])

## Создание прототипа модели

### Разделение выборок

In [ ]:
# Выделение признаков и таргета
X = data_enc_linear.drop(['isChurn'], axis=1)
y = data_enc_linear['isChurn']

# Деление на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=190922)

Проверим, как разделили данные.

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

### Logistic Regression

In [ ]:
log_reg = LogisticRegression()

In [ ]:
C = np.logspace(-4, 4, 50)
penalty = ['l1', 'l2']
solver = ['liblinear']

In [ ]:
parameters = dict(C=C,
                  penalty=penalty,
                  solver=solver,
                  random_state=[190922],
                  class_weight=['balanced'])

In [ ]:
gs = GridSearchCV(log_reg, parameters,
                  cv=3,
                  verbose=2,
                 scoring='roc_auc')

In [ ]:
print(log_reg.get_params().keys())

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
print('Best Penalty:', gs.best_estimator_.get_params()['penalty'])
print('Best C:', gs.best_estimator_.get_params()['C'])
print('Best score: ', gs.best_score_)

Тоже самое на нормализованных данных (`TotalCharged`)

In [ ]:
data_normalized = data_enc_linear.copy()

x = X_train['Days'].values # вернет numpy массив
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(x.reshape(-1, 1))
X_all = data_normalized['Days'].values
data_normalized['Days'] = min_max_scaler.transform(X_all.reshape(-1, 1))

Проверим результат.

In [ ]:
data_normalized.head()

In [ ]:
# Выделение признаков и таргета нормализированных данных
X = data_normalized.drop(['isChurn'], axis=1)
y = data_normalized['isChurn']

# Деление на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=190922)

In [ ]:
gs = GridSearchCV(log_reg, parameters,
                  cv=3,
                  verbose=2,
                 scoring='roc_auc')

gs.fit(X_train, y_train)

In [ ]:
print('Лучший Penalty на нормализованных данных:', gs.best_estimator_.get_params()['penalty'])
print('Лучший C:', gs.best_estimator_.get_params()['C'])
print('Лучшая оценка: ', gs.best_score_)

In [ ]:
columns = ['ROC-AUC-SCORE']
indexes = ['Нормализованные данные', 'Обычные данные']
data = [0.8454946416951317, 0.8458297358116109]

lr_cv = pd.DataFrame(data=data, index=indexes, columns=columns)

lr_cv

Как можно увидеть, результаты на кроссвалидации при обычных данных чуть лучше, чем на нормализованных.

### CatBoostClassifier

In [ ]:
cb = CatBoostClassifier()

In [ ]:
# Выделение признаков и таргета
X = data_enc.drop(['isChurn'], axis=1)
y = data_enc['isChurn']

# Деление на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=190922)

In [ ]:
parameters = {'depth': [1, 3, 5, 7, 10, 13, 15, 17, 20],
             'learning_rate': [.1, .01, .02, .035],
             'iterations': [90, 100, 120, 140, 150, 170, 200],
             'random_state': [190922],
             'loss_function': ['Logloss']}

In [ ]:
grid_cb = GridSearchCV(estimator=cb, param_grid =parameters, cv=3, n_jobs=-1, verbose=2, scoring='roc_auc')
grid_cb.fit(X_train, y_train)

In [ ]:
print(" Результаты CatBoostClassifier " )
print("\n Лучшая оценка из всех параметров:\n", grid_cb.best_score_)
print("\n Лучшие параметры из всех искомых параметров:\n", grid_cb.best_params_)

### KNN метод

In [ ]:
model = KNeighborsClassifier()

In [ ]:
parameters = {'n_neighbors': [3, 5, 7, 10],
             'weights': ['uniform', 'distance'],
             'p': [1, 2]}

In [ ]:
# Выделение признаков и таргета
X = data_enc_linear.drop(['isChurn'], axis=1)
y = data_enc_linear['isChurn']

# Деление на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=190922)

In [ ]:
model_cv = GridSearchCV(estimator=model, param_grid=parameters, cv=3, n_jobs=-1, verbose=2, scoring='roc_auc')
model_cv.fit(X_train, y_train)

In [ ]:
print(" Результаты KNN " )
print("\n Лучшая оценка из всех параметров:\n", model_cv.best_score_)
print("\n Лучшие параметры из всех искомых параметров:\n", model_cv.best_params_)

### Метод случайного леса

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
parameters = {'n_estimators': [10, 30, 50, 70, 90, 100],
             'max_depth': [10, 15, 20],
             'random_state': [190922]}

In [ ]:
# Выделение признаков и таргета
X = data_enc.drop(['isChurn'], axis=1)
y = data_enc['isChurn']

# Деление на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=190922)

In [ ]:
rfc_cv = GridSearchCV(estimator=rfc, param_grid=parameters, n_jobs=-1, verbose=2, scoring='roc_auc')
rfc_cv.fit(X_train, y_train)

In [ ]:
print(" Результаты случайного леса " )
print("\n Лучшая оценка из всех параметров:\n", rfc_cv.best_score_)
print("\n Лучшие параметры из всех искомых параметров:\n", rfc_cv.best_params_)

## Тестирование модели

Из всех моделей, лучшую оценку на кроссвалидации сделала модель CatBoostClassifier. Её проверим на тестовой.

In [10]:
cb = CatBoostClassifier(depth=5, iterations=200, learning_rate=.1, random_state=190922)

cb.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
predict_proba = cb.predict_proba(X_test)[:, 1]

print('ROC-AUC:', roc_auc_score(y_test, predict_proba))

In [ ]:
predictions = cb.predict(X_test)

print('Accuracy:', accuracy_score(y_test, predictions))

Нарисуем ROC-кривую.

In [ ]:
fpr, tpr, _ = roc_curve(y_test, predict_proba)

plt.plot(fpr, tpr)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Можно увидеть, что модель предсказывает не случайно.

Удалось найти модель и подобрать для нее гиперпараметры, которая достигает необходимых метрик.

Итоговая метрика ROC-AUC на тестовой выборке: 0.9080533033108837

In [ ]:
features_importance = cb.get_feature_importance()
# features_importance = sorted(features_importance, reverse=True)

importance = pd.DataFrame(data=features_importance, index=list(X_test.columns), columns=['importance'])

importance.sort_values(by='importance', ascending=False).plot(kind='bar',
                                                              figsize=(16,9),
                                                              title='Важность признаков')

Можно увидеть, что для CatBoost'a есть 4 самых важных признака: кол-во дней, тип подлючения, подключения оптического кабеля и отсутствие интернет служб.

In [ ]:
y_pred = cb.predict(X_test)

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap="coolwarm", fmt='3.0f')
plt.title('Матрица ошибок', y=1.05, size=15)

Модель отлично справляется с предсказанием отрицательного класса (клиент не уйдёт), но с предсказанием ухода клиента делает частые ошибки, но и не угадывает. Модель также чаще предсказывает отрицательный класс, что необходимо для бизнеса: определить, уйдет ли клиент.

## Отчет

Были выполнены практически все пункты плана, в том числе была добавлена еще модель для исследования: CatBoostClassifier. К сожалению, в задуманные сроки удалось уложиться,но возникали ошибки по ходу написания кода, которые надо было исправлять (об этом ниже).

Главная трудность возникала при написании модели CatBoostClassifier, где не получилось подать "грязные" данные на вход из-за `ValueError isChurn is not in List`. Через несколько попыток исправлений решила это обойти с помощью закодированных данных.

Ключевые шаги в работе:
* Изучить таблицы с данными по отдельности
* Поменять типы данных и разобраться с пропусками
* Выделить таргет и создать синтетический признак
* Соединить таблицы
* Провести исследовательский анализ данных
* Провести анализ влияния факторов через аналитический метод (VIF)
* Поиск наилучшей модели
* Тестирование лучшей модели

**Как проходила предобработка данных?**
1. Вывела все таблицы и изучила их структуру
2. Перевела строки с датами в тип данных `datetime`, чтобы можно было с ними работать как с числами
3. При попытке изменении типа данных в колонке общего дохода от клиента, было обнаружено, что датасет собран на момент 2020-02-01
4. В следствие этого пропуски в колонке общего дохода от клиента были заполнены нулями, т.к. клиент не успел еще внести "тотал"
5. Далее создала синтетический признак на основе двух имеющихся: дата начала договора и дата его окончания. Сделано это для выведени "лояльности" клиента. Ведь чем дольше он с компанией, тем меньше шансов его ухода.
6. Соединила таблицы.
7. Пропуски в новом датафрейме заполнила `No`, т.к. отсутствие ID клиента в той или иной таблице подразумевает отсутствие и самой услуги у клиента.
8. Удалила признаки с датами для избежание утечки данных (особенно дата окончания договора).
9. Провела проверку на мультиколлинеарность с помощью VIF, в результате которого было удалено два признака. Данная проверка проводилась для работы с линейными моделями и избежания сильных зависимостей.
10. Кодировала категориальные признаки сочетая два способа: OneHotEncoding и LabelEncoding (в ретроспективе понимаю, что для некоторых признаков надо было сделать OHE, а не LabelEncoding, например в колонке `Gender`)

Для обучения модели использовалась часть старых признаков и два новых. Удалены временные признаки, т.к. это не анализ временных рядов и можно получить утечку данных. Добавлен таргет из даты окончания договора - ушел клиент или нет? И добавлено кол-во дней активного договора из обеих дат (начала и окончания договора), показывающий лояльность клиента.

In [ ]:
importance.sort_values(by='importance', ascending=False).plot(kind='bar',
                                                              figsize=(16,5),
                                                              title='Важность признаков')

После анализа важности признаков можно сделать вывод о ненадобности последних пяти признаков: наличие партнера, наличие иждивенцев, две услуги и способ оплаты через почту. Если бизнес будет использовать модель в будущем, то эти данные не нужно собирать для предсказывания ухода от компании.

In [ ]:
indexes = ['LogisticRegression', 'RandomForestClassifier', 'KNN', 'CatBoostClassifier']
data = [0.845830, 0.8581262210692862, 0.8531185164641819, 0.9280533033108837]

pivot = pd.DataFrame(data=data, index=indexes, columns=['CrossValidation Score'])

Сравнение моделей на кросс-валидации ниже.

In [ ]:
pivot.sort_values(by='CrossValidation Score', ascending=False)

В качестве итоговой модели выбран CatBoostClassifier.

**Оценки итоговой модели** <br>

Accuracy: 0.878637 <br>
ROC-AUC: 0.908053